In [4]:
import pandas as pd
import numpy as np
from scipy.interpolate import interp1d
import matplotlib.pyplot as plt

In [5]:
df = pd.read_excel('Ingram Barge Company Barges.xlsx')
df.drop_duplicates('Barge Number', inplace=True)
df.head()

,Barge Number,Hull Type,Dimensions,Coaming Height,Year Built,Cover Type,Cover Weight,Cubes,Light Draft,AVG TPI,...,"8 ft 6""","9 ft 0""","9 ft 3""","9 ft 6""","9 ft 9""","10 ft 0""","10 ft 3""","10 ft 6""","11 ft 0""","TPI 10'6"" +"
0,ING2065,Box,200x35x13',36,1995,OPEN,NaN,79957,16.38,18.18,...,1557,1667,1722,1776,1831,1886,1941,1996,2106,18.33
1,ING948504,Box,200x35x14',60,1994,OPEN,NaN,87307,17.20,18.21,...,1543,1653,1707,1762,1817,1872,1926,1981,2090,18.21
2,ING2273,Box,200x35x13',36,1998,OPEN,NaN,79506,16.75,18.21,...,1561,1671,1725,1780,1835,1889,1944,1999,2108,18.21
3,ING2406,Box,200x35x13',36,1997,OPEN,NaN,84389,18.25,18.26,...,1527,1637,1692,1746,1801,1855,1910,1965,2074,18.17
4,ING2161,Rake,200x35x13',36,1997,OPEN,NaN,78811,17.00,17.22,...,1452,1558,1612,1665,1718,1772,1826,1879,1987,18.08


Let's extract the hull depth from the `Dimensions` column.

In [6]:
df['Hull Depth'] = df['Dimensions'].apply(lambda x: int(x[-3:-1]))
df.head()

,Barge Number,Hull Type,Dimensions,Coaming Height,Year Built,Cover Type,Cover Weight,Cubes,Light Draft,AVG TPI,...,"9 ft 0""","9 ft 3""","9 ft 6""","9 ft 9""","10 ft 0""","10 ft 3""","10 ft 6""","11 ft 0""","TPI 10'6"" +",Hull Depth
0,ING2065,Box,200x35x13',36,1995,OPEN,NaN,79957,16.38,18.18,...,1667,1722,1776,1831,1886,1941,1996,2106,18.33,13
1,ING948504,Box,200x35x14',60,1994,OPEN,NaN,87307,17.20,18.21,...,1653,1707,1762,1817,1872,1926,1981,2090,18.21,14
2,ING2273,Box,200x35x13',36,1998,OPEN,NaN,79506,16.75,18.21,...,1671,1725,1780,1835,1889,1944,1999,2108,18.21,13
3,ING2406,Box,200x35x13',36,1997,OPEN,NaN,84389,18.25,18.26,...,1637,1692,1746,1801,1855,1910,1965,2074,18.17,13
4,ING2161,Rake,200x35x13',36,1997,OPEN,NaN,78811,17.00,17.22,...,1558,1612,1665,1718,1772,1826,1879,1987,18.08,13


In [7]:
drafts = df.loc[:,'8 feet. 0"':'11 ft 0"'].columns.to_list()
columns_to_keep = ['Barge Number', 'Hull Type', 'Hull Depth'] + drafts
barges = df[columns_to_keep].copy()
barges.set_index('Barge Number', inplace=True)
barges.head()

,Hull Type,Hull Depth,"8 feet. 0""","8 ft 6""","9 ft 0""","9 ft 3""","9 ft 6""","9 ft 9""","10 ft 0""","10 ft 3""","10 ft 6""","11 ft 0"""
Barge Number,,,,,,,,,,,,
ING2065,Box,13,1448,1557,1667,1722,1776,1831,1886,1941,1996,2106
ING948504,Box,14,1434,1543,1653,1707,1762,1817,1872,1926,1981,2090
ING2273,Box,13,1452,1561,1671,1725,1780,1835,1889,1944,1999,2108
ING2406,Box,13,1420,1527,1637,1692,1746,1801,1855,1910,1965,2074
ING2161,Rake,13,1346,1452,1558,1612,1665,1718,1772,1826,1879,1987


In [12]:
drafts_vals = [s.replace('"','').strip().split() for s in drafts]
draft_vals = np.array([int(s[0]) + int(s[-1])/12 for s in drafts_vals])

Below is a function to export `Pandas.DataFrame` as an HTML table with a titile.

In [9]:
import pandas as pd
import pandas.io.formats.style

def write_to_html_file(df, title='', filename='out.html'):
    '''
    Write an entire dataframe to an HTML file with nice formatting.
    '''

    result = '''
<html>
<head>
<style>

    h2 {
        text-align: center;
        font-family: Helvetica, Arial, sans-serif;
    }
    table { 
        margin-left: auto;
        margin-right: auto;
    }
    table, th, td {
        border: 1px solid black;
        border-collapse: collapse;
    }
    th, td {
        padding: 5px;
        text-align: center;
        font-family: Helvetica, Arial, sans-serif;
        font-size: 90%;
    }
    table tbody tr:hover {
        background-color: #dddddd;
    }
    .wide {
        width: 90%; 
    }

</style>
</head>
<body>
    '''
    result += '<h2> %s </h2>\n' % title
    if type(df) == pd.io.formats.style.Styler:
        result += df.render()
    else:
        result += df.to_html(classes='wide', escape=False)
    result += '''
</body>
</html>
'''
    with open(filename, 'w') as f:
        f.write(result)

Generating Standard Table and Plots.

In [10]:
import matplotlib
for barge_name in barges.index:

    barge = barges.loc[barge_name]
    f = interp1d(x = (barge['Hull Depth'] - draft_vals), y= barge[drafts],
                 bounds_error=False, fill_value="extrapolate")

    table = pd.DataFrame(index=np.arange(1,12), columns=np.linspace(0, 0.9, 10))
    table.index.name = 'Freeboard (ft)'

    for i in table.index:
        for j in table.columns:
            table.loc[i,j] = np.around(f(i+j)) if f(i+j)>0 and f(i+j) < max(barge[drafts]) else np.nan

    table.columns = [str(x)[:3] for x in table.columns]
    title = f'Barge {barge_name}, Hull Depth: {barge["Hull Depth"]}\', Hull Type: {barge["Hull Type"]}'
    write_to_html_file(table, title, f'{barge_name}.html')
    
    plt.figure(figsize=(6,4))
    plt.scatter(barge[drafts], draft_vals, s=20, label='Table Values', alpha=0.5, c='black')
    def get_tonnage(arr):
        return [np.around(f(num)) if (f(num)>0 and f(num) <= max(barge[drafts])) else np.nan for num in arr]
    plt.plot(get_tonnage(np.arange(1,13)) , barge['Hull Depth']-np.arange(1,13), ls='--', label='Interpolation', alpha=0.5, 
             zorder=0)
    plt.legend()
    ax = plt.gca()
    ax.set_ylabel('Draft (ft)')
    secay = ax.secondary_yaxis('right', functions=(lambda x: barge['Hull Depth']-x, lambda x: barge['Hull Depth']-x))
    ax.get_xaxis().set_major_formatter(
    matplotlib.ticker.FuncFormatter(lambda x, p: format(int(x), ',')))
    secay.set_ylabel('Freeboard (ft)')
    plt.title(title)
    plt.tight_layout(pad=2)
    plt.savefig(f'{barge_name}.png', dpi=400)
    plt.close()
    